In [ ]:
###
# This code is for evaluating the performance of YOLO.
###

In [ ]:
from __future__ import print_function

import glob, os, random, torch, timm, shutil, pickle, time, yaml, json, gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
import itertools

from PIL import Image, ImageEnhance, ImageOps, ImageDraw
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from pathlib import Path
from pprint import pprint
from tempfile import TemporaryDirectory
from ultralytics import YOLO

os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

seed = 0

Image.MAX_IMAGE_PIXELS = None
%matplotlib inline



In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [ ]:
##########

In [ ]:
DEVICE = 2
EPOCH = 500
BATCH = 1
IOU = 0.5
CONF = 0.001

In [ ]:
#####

In [ ]:
models_d = '/PATH/TO/YOUR/WEIGHTS'
models_f = sorted(glob.glob(models_d + '/*.pt'))
models_f

In [ ]:
for i in models_f:
    DATA = "/PATH/TO/YOUR/YAML"
    model = YOLO(i)
    metrics = model.val(data=DATA, imgsz=640, batch=BATCH, conf=CONF, iou=IOU, device=DEVICE,
                            project='PAS_glo', name=os.path.splitext(DATA)[0] + '___on___' + os.path.basename(os.path.splitext(i)[0]))
    with open(os.path.splitext(DATA)[0] + '___on___' + os.path.basename(os.path.splitext(i)[0]) + '.pkl', 'wb') as file:
        pickle.dump(metrics.box, file)
        
    del model
    gc.collect()
    